# Without sWeights: `common_Dst_constr_B0toDstDsWithmu2Free_BDT-1.25`

# Fit the complete data
    
TWO TASKS: (\# iterations: $\approx$1000)
- **Mode 1**: The fixed (=tail) parameters of the $D^*\pi\pi\pi$ are randomly changed within their uncertainty.
For the fixed parameters $p\in\{\alpha_L, \alpha_R, n_L, n_R\}$ of uncertainty $\Delta p$ and nominal value $p_0$:
$$p=p_0+\mathcal{N}(\mu=0, \sigma = \Delta p)$$
    The ratio of yield $\frac{n_{B^0 \to D^{*-} K^+ \pi^+ \pi^-}}{n_{B^0 \to D^{*-} \pi^+ \pi^+ \pi^-}}$ and the parameters of the $D^*Kpipi$ fit are still fixed. The fit on is performed again with those values.
    The results of the fits are stored in a JSON file, in particular the signal yield $n_{sig}$.

- **Mode 2**: The same is done with the ratio of yield $\frac{n_{B^0 \to D^{*-} K^+ \pi^+ \pi^-}}{n_{B^0 \to D^{*-} \pi^+ \pi^+ \pi^-}}$ and the parameters of the $D^*Kpipi$ fit. A  fit is performed on $D^*\pi\pi\pi$.
The results of the fits are stored in a JSON file, in particular the signal yield $n_{sig}$.


Fit on $m(B^0_s)$ with:
- Signal $B^0 \to D^{*-} \pi^+ \pi^+ \pi^-$ (sum of two Crystall ball PDF)
- Background $B^0 \to D^{*-} D_s^+$ from the other fit of the simultaneous fit
- Background with a misidentified pion: $B^0 \to D^{*-} K^+ \pi^+ \pi^-$ (sum of two crystall ball PDF)
- Combinatorial background (exponential)

In [1]:
data_name_MC = 'MC_constr_opposedCB_sharedMean'
data_name_KPiPi = "data_KPiPi_constr"
BDT_name = 'gradient'

data_name = 'common_B0TODst3pi_Dst_constr'
data_name_B0toDstDs = 'common_B0TODstDs_Dst_constr'
data_name_Dstto3pi = 'common_DstTO3pi'

BDT = -1.25

In [3]:
branch_constr = 'Dst_constr_B0_M'
low = 5150.
high = 5545.

## Import

In [11]:
import zfit
import timeit
import pandas as pd
import numpy as np


from bd2dst3pi.locations import loc
from bd2dst3pi.definition import (
    years, magnets, latex_params, 
    latex_params_B0toDstDs, latex_params_KPiPi
)
from bd2dst3pi.pandas_root import load_data, add_constr_Dst

import sys
sys.path.append(loc.ROOT + 'library/')

from HEA.plot import plot_hist_auto, plot_hist, plot_hist2d_auto
from HEA.plot.fit import plot_hist_fit_auto
from HEA.fit import (
    json_to_latex_table, 
    retrieve_params, 
    show_latex_table,
    get_params_without_BDT
)

from HEA.fit.fit import (
    launch_fit, save_params, 
    define_zparams, sum_crystalball_or_gaussian, 
    check_fit
)


import HEA.plot.tools as pt
from HEA.tools.serial import dump_pickle
from HEA.pandas_root import save_root, load_saved_root

In [12]:
latex_params_KPiPi_print = {}
for key, value in latex_params_KPiPi.items():
    latex_params_KPiPi_print[key+'_Kpipi'] = latex_params_KPiPi[key]

## Load

### Json files

In [13]:
data_BDT_name = f"{data_name}_BDT{BDT}"
data_name_B0toDstDs_BDT = f"{data_name_B0toDstDs}_BDT{BDT}"

# Fixed alphaL, alphaR and nL
common_params = retrieve_params(data_name, folder_name=data_name)
# Fixed nR
MC_params = retrieve_params(data_name_MC, folder_name='MC')

# B0->DstKPiPi
Kpipi_params = retrieve_params(data_name_KPiPi, folder_name='data_KPiPi') # already in MeV 

# common with BDT cut (for startning values of the fit)
pipipi_params = get_params_without_BDT(retrieve_params(data_BDT_name, folder_name=data_name), True)
# B0->DstDs with BDT cut (for the B0->DstDs PDF shape)
DstDs_params = get_params_without_BDT(retrieve_params(data_name_B0toDstDs_BDT, folder_name=data_name_B0toDstDs), True) 

Load 
- `f_params_dict['1']` - the fixed parameters that will be changed in **mode 1**
- `f_params_dict['2']`  - the fixed parameters that will be changed in **mode 2**

In [14]:
f_params_dict = {}
f_params_dict['1'] = ['alphaL', 'alphaR', 'nL', 'nR']
f_params_dict['2'] = ['r_bkgK', 
                      'muL_Kpipi', 'muR_Kpipi', 'nL_Kpipi', 'nR_Kpipi',
                      'alphaL_Kpipi', 'alphaR_Kpipi',
                      'sigmaL_Kpipi', 'sigmaR_Kpipi', 'fraction_Kpipi']


In [15]:
f_params = {} # fixed parameters

for param in 'alphaL', 'alphaR', 'nL', 'r_bkgK':
    f_params[param] = {}
    f_params[param]['val'] = common_params[param] # nominal value
    f_params[param]['std'] = common_params[param + '_err'] # standard deviation

f_params['nR'] = {}
f_params['nR']['val'] = MC_params['nR_MC']
f_params['nR']['std']  = MC_params['nR_MC_err']

for param in 'muL', 'muR', 'nL', 'nR', 'alphaR', 'alphaL', 'sigmaL', 'sigmaR', 'fraction':
    f_params[param+'_Kpipi'] = {}
    f_params[param+'_Kpipi']['val'] = Kpipi_params[param]
    f_params[param+'_Kpipi']['std'] = Kpipi_params[param+'_err']


In [16]:
df = load_saved_root('common_'+BDT_name, folder_name='common', vars=['B0_M', 'Dst_M', 'BDT'])
df = df.query(f"BDT > {BDT}")
df = add_constr_Dst(df)

Loading /afs/cern.ch/user/c/correian/bd2dst3pi/output//root/common/common_gradient.root


In [17]:
def same_sign(a, b):
    return (a>0 and b>0) or (a<0 and b<0)

## Redo some fits

In [22]:
mode = 1
indexes = [580]
show_index_list = []

# mode = 2
# indexes = [56, 110, 182, 220, 307, 319, 406, 409, 604, 631, 773, 786, 904, 906, 1023, 1063, 1078, 1095]

m = str(mode)

In [23]:
for i in indexes:
    verbose = i in show_index_list
    name_iter = data_BDT_name + f'u{mode};{i}'

    print('iter:', i)

    r_params = retrieve_params(f"{data_BDT_name}u{m};{i}", folder_name=data_BDT_name)['r_params']


    ### FIT ON DstPiPiPi ============================================================     
    ## Starting point of the fit ...............

    n_events_B0_M = len(df.query(f'{branch_constr} > {low} and {branch_constr} < {high}'))   

    initial_values = {
        # Signal        
        'mu'       :{ 'value': pipipi_params['mu']           , 'low': 5279.-3.                      , 'high': 5279.+3.             , 'floating': True},
        'sigmaL'   :{ 'value': pipipi_params['sigmaL']       , 'low': 5.                            , 'high': 20.                 , 'floating': True},
        'sigmaR'   :{ 'value': pipipi_params['sigmaR']       , 'low': 10.                           , 'high': 30.               , 'floating': True},
        'frac'     :{ 'value': pipipi_params['frac']         , 'low': 0.1                           , 'high': 1.                   , 'floating': True},
        'n_sig'    :{ 'value': pipipi_params['n_sig']        , 'low': pipipi_params['n_sig']*0.1    , 'high': n_events_B0_M        , 'floating': True},
        # Combinatorial background        
        'lambda'   :{ 'value': pipipi_params['lambda']       , 'low': -1.                           , 'high': 0.                   , 'floating': True},
        'n_bkg'    :{ 'value': pipipi_params['n_bkg']        , 'low': 0                             , 'high': n_events_B0_M        , 'floating': True},
        # Background decay D*Kpipi    
    }

    # Fixed parameters:
    for param, value_param in r_params.items():
        initial_values[param] = {}
        initial_values[param]['value'] = value_param
        initial_values[param]['floating'] = False

    # B0 -> D* Ds distribution
    for vari in 'mu', 'sigma', 'n_sig' :
        initial_values[f'{vari}_Ds'] = {'value': DstDs_params[f'{vari}_DstDs'], 'low': None, 'high': None, 'floating': False}

    ## Observable and variables .................
    obs = zfit.Space("x", limits=(low, high))

    zparams = define_zparams(initial_values, num=i)

    mult = lambda r, n: r*n
    zparams["n_bkgK"] = zfit.ComposedParameter(f"n_bkgK{i}", mult, params=[zparams['r_bkgK'],zparams['n_sig']])    

    ## PDFs .....................................
    model_S, crystalBallL_S, crystalBallR_S = sum_crystalball_or_gaussian(zparams['mu'], zparams['mu'],
                                                              zparams['sigmaL'], zparams['sigmaR'],
                                                              zparams['frac'], obs,                                                          
                                                              zparams['alphaL'], zparams['alphaR'], 
                                                              zparams['nL'], zparams['nR']
                                                             )


    gaussian_Ds = zfit.pdf.Gauss(zparams['mu_Ds'],zparams['sigma_Ds'],obs=obs)

    model_K, crystalBallL_K, crystalBallR_K = sum_crystalball_or_gaussian(zparams['muL_Kpipi'], zparams['muR_Kpipi'], 
                                                              zparams['sigmaL_Kpipi'], zparams['sigmaR_Kpipi'],
                                                              zparams['fraction_Kpipi'], obs,
                                                              zparams['alphaL_Kpipi'], zparams['alphaR_Kpipi'], 
                                                              zparams['nL_Kpipi'], zparams['nR_Kpipi'])

    exponential = zfit.pdf.Exponential(zparams['lambda'], obs=obs)


    model_S_ext = model_S.create_extended(zparams['n_sig'])
    exp_ext = exponential.create_extended(zparams['n_bkg'])
    gaussian_Ds_ext = gaussian_Ds.create_extended(zparams['n_sig_Ds'])
    model_K_ext = model_K.create_extended(zparams['n_bkgK'])

    model = zfit.pdf.SumPDF([model_S_ext, gaussian_Ds_ext, model_K_ext, exp_ext]) # gauss is folded by frac
    data = zfit.Data.from_pandas(df[branch_constr], obs=obs)

    ## Do the fit ...............................
    start = timeit.default_timer()
    result, params = launch_fit(model, data, extended=True, verbose=verbose)
    result, params = launch_fit(model, data, extended=True, verbose=verbose)
    stop = timeit.default_timer()
    print('Time to do the fit: ', stop - start) 
    fit_ok = check_fit(result)

    ## Plot and save the fit ....................
    if verbose:
        data_name_plot = data_name + f'{i}_details'
        plot_hist_fit_auto(df, branch_constr, models = [model,[model_S_ext, crystalBallL_S, crystalBallR_S], gaussian_Ds_ext,
                                                  model_K_ext,exp_ext],
                               models_names = ["", '\n $B^0\\to D^{*-} 3\pi$','\n $B^0\\to D^{*-}D_s^+$',
                                              '\n $B^0\\to D^{*-} K^+ \pi^+\pi^-$','\n Combinatorial'],
                               obs=obs, n_bins = 100,bar_mode = False,
                               data_name = data_name_plot, folder_name=data_name,
                               colors=['b','g','r','saddlebrown','y'],
                                params=params,latex_params=latex_params,
                               fontsize_leg=13.5, show_chi2=True,
                               colWidths=[0.06,0.01,0.05,0.06], 
                              )

        data_name_plot = data_name + f'{i}'
        plot_hist_fit_auto(df,branch_constr, models = [model, model_S_ext, gaussian_Ds_ext, model_K_ext,exp_ext],
                              models_names = ["", '\n $B^0\\to D^{*-} 3\pi$','\n $B^0\\to D^{*-}D_s^+$',
                                              '\n $B^0\\to D^{*-} K^+ \pi^+\pi^-$','\n Combinatorial'],
                               obs=obs, n_bins = 100,bar_mode = False,
                               data_name = data_name_plot, folder_name=data_name,
                               colors=['b','g','r','saddlebrown','y'], loc_leg='best',
                               fontsize_leg=20., pos_text_LHC=[0.95, 0.30, 'right'],
                              )



    save_params(params, name_iter, True, 
                {'fit_ok': fit_ok, 'info_fit':str(result.info['original']), 'info_params': str(params),
                 'r_params': r_params}, 
                folder_name=data_BDT_name)

iter: 580
Time to do the fit:  10.361111823935062
